In [1]:
import csv
from csv import writer
import pandas as pd

In [2]:
from openai import OpenAI

In [3]:
# The contents of this cell are adopted from
# https://github.com/UG-Team-Data-Science/ai-in-edu/blob/main/utils.py
# created by Michiel van der Ree.

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
model = "meta-llama/Meta-Llama-3-8B-Instruct"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

def format_history(message, history, system_prompt):
    chat_history = [{"role": "system", "content": system_prompt}]
    for query, response in history:
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})
    chat_history.append({"role": "user", "content": message})
    return chat_history

def prompt_model(
    message,
    history=None,
    system_prompt="You are a powerful annotator of event triggers and non-event triggers",
):
    chat_history = format_history(message, history if history else [], system_prompt)
    response = client.chat.completions.create(model=model, messages=chat_history)
    return response.choices[0].message.content

In [2]:
df1 = pd.read_csv('MNT_TEST_list_of_sentences.csv')

In [3]:
df1.head(10)

,Unnamed: 0,sentences
0,0,Northrop Grumman and Airbus parent EADS defeat...
1,1,"March 01, 2008"
2,2,Northrop Grumman and Airbus parent company Eur...
3,3,It is the biggest contract of its kind since t...
4,4,That contract was fought for between Boeing an...
5,5,JSA Research defence analysist Paul Nesbit sai...
6,6,"December 20, 2007"
7,7,"Jusman Syafii Djamal, minister of transport fo..."
8,8,It is a direct response to the decision of the...
9,9,The EU made the decision to ban the Indonesian...


In [4]:
mysentences = df1['sentences'].tolist()
print(len(mysentences))

1115


In [12]:
# create a csv file
fields = ['id', 'sentence', 'prompt', 'LLM_response']
filename = 'MNT_TEST_zero_shot_round2.csv'
# writing to csv file
with open(filename, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)
    # writing the fields
    csvwriter.writerow(fields)

In [13]:
df = pd.read_csv('MNT_TEST_zero_shot_round2.csv')

In [14]:
df

,id,sentence,prompt,LLM_response


In [15]:
def outputs(sentences):
    for sentence in sentences:
        ids = id(sentence)
        sentence = sentence
        prompt = f"""Given the sentence between the brackets <{sentence}>, annotate each token of the sentence, including punctuation marks, as an event trigger with the tag <EVENT> or as a non-event trigger with the tag <O>.
Return exclusively the annotated results token by token. Provide each token without brackets followed by its associated annotation tag in the same line. Provide each subsequent token next to its associated annotation tag in a new line.
Do not add any extra introduction, notes or explanation.
"""
        output = prompt_model(prompt)
        df.loc[len(df.index)] = [ids, sentence, prompt, output]
    return

In [16]:
outputs(mysentences)

In [17]:
df

,id,sentence,prompt,LLM_response
0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...
1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>..."
2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...
3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...
4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...
...,...,...,...,...
1110,140538816665664,"September 18, 2008",Given the sentence between the brackets <Septe...,September <EVENT>\n18 <O>\n2008 <EVENT>
1111,140538816442608,Stock indexes around the globe reacted favorab...,Given the sentence between the brackets <Stock...,Stock <EVENT>\n<EVENT>\nindexes <O>\naround <O...
1112,140538816338736,"At 0800 UTC, the Bank of Canada, the Bank of E...",Given the sentence between the brackets <At 08...,"At <EVENT>\n<EVENT>\n0800 <O>\nUTC <O>\n, <O>\..."
1113,140538816712528,"The credit arrangements, also known as `` swap...",Given the sentence between the brackets <The c...,The credit arrangements \n<EVENT>\nalso \n<O>\...


In [18]:
df.head(10)

,id,sentence,prompt,LLM_response
0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...
1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>..."
2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...
3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...
4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...
5,140538820797424,JSA Research defence analysist Paul Nesbit sai...,Given the sentence between the brackets <JSA R...,JSA <O>\nResearch <O>\ndefence <O>\nanalysist ...
6,140541903195792,"December 20, 2007",Given the sentence between the brackets <Decem...,December <EVENT>\n20 <EVENT>\n2007 <EVENT>\n> <O>
7,140538816401616,"Jusman Syafii Djamal, minister of transport fo...",Given the sentence between the brackets <Jusma...,Jusman <O>\nSyafii <O>\nDjamal <O>\nminister <...
8,140538818115232,It is a direct response to the decision of the...,Given the sentence between the brackets <It is...,It <O>\nis <O>\na <O>\ndirect <O>\nresponse <E...
9,140538816301616,The EU made the decision to ban the Indonesian...,Given the sentence between the brackets <The E...,The\n<EVENT>\nEU\n<EVENT>\nmade\n<EVENT>\nthe\...


In [19]:
df.to_csv('MNT_TEST_zero_shot_round2_withtabs.csv', sep = "\t")

In [20]:
df.to_csv('MNT_TEST_zero_shot_round2_withcommas.csv')

In [21]:
df2 = pd.read_csv('MNT_TEST_zero_shot_round2_withtabs.csv', sep = "\t")
df2.head()

,Unnamed: 0,id,sentence,prompt,LLM_response
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>..."
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...


In [22]:
df = pd.read_csv('MNT_TEST_zero_shot_round2_withcommas.csv')
df.head()

,Unnamed: 0,id,sentence,prompt,LLM_response
0,0,140538831478848,Northrop Grumman and Airbus parent EADS defeat...,Given the sentence between the brackets <North...,North <O>\nGrumman <O>\nand <O>\nAirbus <O>\np...
1,1,140538816684656,"March 01, 2008",Given the sentence between the brackets <March...,"March <EVENT>\n01 <EVENT>\n2008 <EVENT>\n, <O>..."
2,2,140538820797072,Northrop Grumman and Airbus parent company Eur...,Given the sentence between the brackets <North...,Northrop Grumman <O>\n<EVENT>\nand <O>\nAirbus...
3,3,140538817786288,It is the biggest contract of its kind since t...,Given the sentence between the brackets <It is...,It <EVENT>\nis <O>\nthe <O>\nbiggest <O>\ncont...
4,4,140538816323440,That contract was fought for between Boeing an...,Given the sentence between the brackets <That ...,That EVENT\nwas EVENT\ncontract EVENT\nfought ...
